[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/witchapong/build-ai-based-applications/blob/main/tabular/1_collect_data.ipynb)

# Stock Price Prediction using ML model
In this session, we'll learn how to build a ML model for predicting **%change of stock prices of the next day** of stocks in SET index (Stock Exchange of Thailand). Thus, we should be able to use the prediction to buy stocks that are going up the next day, make profits, and hopefully get rich!

This session is divided into the following 5 notebooks.
1. `1_collect_data.ipynb` (current notebook)
2. `2_eda.ipynb`
3. `3_features_prep.ipynb`
4. `4_make_prediction.ipynb`
5. `5_evaluation.ipynb`

In [1]:
# # clone Github Repo
# !git clone https://github.com/witchapong/build-ai-based-applications.git

Cloning into 'build-ai-based-applications'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 66 (delta 38), reused 26 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (66/66), 1.18 MiB | 10.21 MiB/s, done.
Resolving deltas: 100% (38/38), done.


In [2]:
# # mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
# # copy data folders from Github Repo to Google Drive
# !mkdir -p /content/drive/MyDrive/build-ai-based-applications/data && cp -rp build-ai-based-applications/tabular/data/* /content/drive/MyDrive/build-ai-based-applications/data

In [4]:
from pathlib import Path

# running on Colab
DATA_DIR = Path("/content/drive/MyDrive/build-ai-based-applications/")

# # running on local
# DATA_DIR = Path(".")

# Load stock data using `YahooFinance`
In this notebook, we will download and store the following data
1. Daily price - end of day information of stocks i.e. Open, High, Low, Close, Volume
2. Company information - sector and industry of stocks
3. Annual income statement - profitability of stocks

In [5]:
import pandas as pd
import yfinance as yf
from yfinance.exceptions import YFRateLimitError
from tqdm.notebook import tqdm
import time

## Load symbol list

In [6]:
# read symbols list within SET index from file
symbols = pd.read_csv(DATA_DIR / "data/SET_symbols_20241230.csv")

## Daily price

In [7]:
# configurations
interval = "1d"
start = "2020-01-01"
end = "2024-12-31"

In [8]:
price_df = pd.DataFrame()

for symbol in tqdm(symbols["symbol"]):

    success = False

    while(not success):
      try:
        yf_ticker = yf.Ticker(f"{symbol}.BK")
        price_df = pd.concat([price_df, yf_ticker.history(interval=interval, start=start, end=end).assign(symbol=symbol)])
        success = True

      except Exception as e:
        print(f"{e} occurred. Fail to fetch {symbol}. Waiting for 5 second...")
        time.sleep(5)

# check-point
price_df.to_csv(DATA_DIR / "data/set_price.csv")


  0%|          | 0/925 [00:00<?, ?it/s]

ERROR:yfinance:Failed to get ticker 'AEONTS.BK' reason: Expecting value: line 1 column 1 (char 0)
ERROR:yfinance:$AEONTS.BK: possibly delisted; no timezone found
<ipython-input-8-763f81aa979c>:10: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  price_df = pd.concat([price_df, yf_ticker.history(interval=interval, start=start, end=end).assign(symbol=symbol)])
ERROR:yfinance:Failed to get ticker 'AF.BK' reason: Expecting value: line 1 column 1 (char 0)
ERROR:yfinance:$AF.BK: possibly delisted; no timezone found
<ipython-input-8-763f81aa979c>:10: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entrie

In [9]:
price_df = pd.read_csv(DATA_DIR / "data/set_price.csv")

In [10]:
price_df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,symbol,Capital Gains,Adj Close
0,2022-10-03 00:00:00+07:00,7.10,10.20,7.10,10.20,559465900.0,0.0,0.0,24CS,NaN,NaN
1,2022-10-04 00:00:00+07:00,10.70,11.10,7.15,7.15,330707400.0,0.0,0.0,24CS,NaN,NaN
2,2022-10-05 00:00:00+07:00,5.85,6.45,5.05,5.15,361028900.0,0.0,0.0,24CS,NaN,NaN
3,2022-10-06 00:00:00+07:00,5.40,5.45,4.70,5.20,232679200.0,0.0,0.0,24CS,NaN,NaN
4,2022-10-07 00:00:00+07:00,5.10,5.15,4.76,5.00,131778400.0,0.0,0.0,24CS,NaN,NaN


In [11]:
price_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 952712 entries, 0 to 952711
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           952712 non-null  object 
 1   Open           952695 non-null  float64
 2   High           952695 non-null  float64
 3   Low            952695 non-null  float64
 4   Close          952695 non-null  float64
 5   Volume         952712 non-null  float64
 6   Dividends      952712 non-null  float64
 7   Stock Splits   952712 non-null  float64
 8   symbol         952712 non-null  object 
 9   Capital Gains  14532 non-null   float64
 10  Adj Close      0 non-null       float64
dtypes: float64(9), object(2)
memory usage: 80.0+ MB


## Daily price of SET (benchmark)

In [12]:
# Download data for a specified date range (e.g., last 5 years)
set_data = yf.download("^SET.BK", start=start, end=end, interval=interval)

[*********************100%***********************]  1 of 1 completed


In [13]:
set_data.head()

Price,Close,High,Low,Open,Volume
Ticker,^SET.BK,^SET.BK,^SET.BK,^SET.BK,^SET.BK
Date,,,,,
2020-01-02,1595.819946,1597.920044,1583.180054,1584.349976,3442200
2020-01-03,1594.969971,1604.430054,1592.900024,1596.949951,3251500
2020-01-06,1568.500000,1585.560059,1565.930054,1584.130005,4116100
2020-01-07,1585.229980,1585.439941,1570.040039,1578.520020,3201300
2020-01-08,1559.270020,1572.030029,1555.750000,1569.819946,3619500


In [14]:
set_data.columns = [col1 for col1, col2 in set_data.columns]

In [15]:
set_data.head()

,Close,High,Low,Open,Volume
Date,,,,,
2020-01-02,1595.819946,1597.920044,1583.180054,1584.349976,3442200
2020-01-03,1594.969971,1604.430054,1592.900024,1596.949951,3251500
2020-01-06,1568.500000,1585.560059,1565.930054,1584.130005,4116100
2020-01-07,1585.229980,1585.439941,1570.040039,1578.520020,3201300
2020-01-08,1559.270020,1572.030029,1555.750000,1569.819946,3619500


In [16]:
set_data.reset_index(inplace=True)

In [17]:
set_data.to_csv(DATA_DIR / "data/set_price_index.csv", index=False)

## Company information

In [18]:
# get company info
company_info_df = pd.DataFrame()

for symbol in tqdm(symbols["symbol"]):

  success = False

  while(not success):

    try:
      ticker = yf.Ticker(f"{symbol}.BK")
      symbol_info = ticker.info

      if len(symbol_info) > 1:
              company_info_df = pd.concat([company_info_df, pd.DataFrame({"symbol": [symbol], "industry": [symbol_info.get("industry")], "sector": [symbol_info.get("sector")]})])
      success = True

    except Exception as e:
            print(f"\"{e}\" occurred. Fail to fetch {symbol}. Waiting for 5 second...")
            time.sleep(5)

company_info_df.to_csv(DATA_DIR / "data/set_company_info.csv", index=False)


  0%|          | 0/925 [00:00<?, ?it/s]

"Too Many Requests. Rate limited. Try after a while." occurred. Fail to fetch PERM. Waiting for 5 second...
"Too Many Requests. Rate limited. Try after a while." occurred. Fail to fetch PERM. Waiting for 5 second...
"Too Many Requests. Rate limited. Try after a while." occurred. Fail to fetch PERM. Waiting for 5 second...
"Too Many Requests. Rate limited. Try after a while." occurred. Fail to fetch PERM. Waiting for 5 second...
"Too Many Requests. Rate limited. Try after a while." occurred. Fail to fetch PERM. Waiting for 5 second...
"Too Many Requests. Rate limited. Try after a while." occurred. Fail to fetch PERM. Waiting for 5 second...
"Too Many Requests. Rate limited. Try after a while." occurred. Fail to fetch PERM. Waiting for 5 second...
"Too Many Requests. Rate limited. Try after a while." occurred. Fail to fetch PERM. Waiting for 5 second...
"Too Many Requests. Rate limited. Try after a while." occurred. Fail to fetch PERM. Waiting for 5 second...
"Too Many Requests. Rate lim

ERROR:yfinance:401 Client Error: Unauthorized for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SSPF.BK?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=SSPF.BK&crumb=Edge%3A+Too+Many+Requests


"Too Many Requests. Rate limited. Try after a while." occurred. Fail to fetch WAVE. Waiting for 5 second...
"Too Many Requests. Rate limited. Try after a while." occurred. Fail to fetch WAVE. Waiting for 5 second...
"Too Many Requests. Rate limited. Try after a while." occurred. Fail to fetch WAVE. Waiting for 5 second...
"Too Many Requests. Rate limited. Try after a while." occurred. Fail to fetch WAVE. Waiting for 5 second...
"Too Many Requests. Rate limited. Try after a while." occurred. Fail to fetch WAVE. Waiting for 5 second...
"Too Many Requests. Rate limited. Try after a while." occurred. Fail to fetch WAVE. Waiting for 5 second...
"Too Many Requests. Rate limited. Try after a while." occurred. Fail to fetch WAVE. Waiting for 5 second...
"Too Many Requests. Rate limited. Try after a while." occurred. Fail to fetch WAVE. Waiting for 5 second...
"Too Many Requests. Rate limited. Try after a while." occurred. Fail to fetch WAVE. Waiting for 5 second...
"Too Many Requests. Rate lim

In [19]:
company_info_df = pd.read_csv(DATA_DIR / "data/set_company_info.csv")

In [20]:
company_info_df.head()

,symbol,industry,sector
0,24CS,Building Products & Equipment,Industrials
1,2S,Steel,Basic Materials
2,3BBIF,NaN,NaN
3,A,Real Estate - Development,Real Estate
4,A5,Real Estate - Development,Real Estate


In [21]:
company_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 924 entries, 0 to 923
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   symbol    924 non-null    object
 1   industry  857 non-null    object
 2   sector    857 non-null    object
dtypes: object(3)
memory usage: 21.8+ KB


## Income statement

In [22]:
incm_stmt_df = pd.DataFrame()

for symbol in tqdm(symbols["symbol"]):

    success = False

    while(not success):

      try:
          ticker = yf.Ticker(f"{symbol}.BK")
          _incm_stmt_df = ticker.income_stmt

          if len(_incm_stmt_df) > 0:

              _incm_stmt_df = _incm_stmt_df.T
              _incm_stmt_df = _incm_stmt_df.assign(symbol=symbol).set_index("symbol", append=True)
              _incm_stmt_df.index.names = ["date", "symbol"]

              incm_stmt_df = pd.concat([incm_stmt_df, _incm_stmt_df])

          success = True
      except Exception as e:
        print(f"\"{e}\" occurred. Fail to fetch {symbol}. Waiting for 5 second...")
        time.sleep(5)

incm_stmt_df.reset_index(inplace=True)

# store the data
incm_stmt_df.to_csv(DATA_DIR / "data/set_incm_stmt.csv", index=False)

  0%|          | 0/925 [00:00<?, ?it/s]

In [23]:
incm_stmt_df = pd.read_csv(DATA_DIR / "data/set_incm_stmt.csv")

In [24]:
incm_stmt_df.head()

,date,symbol,Tax Effect Of Unusual Items,Tax Rate For Calcs,Normalized EBITDA,Total Unusual Items,Total Unusual Items Excluding Goodwill,Net Income From Continuing Operation Net Minority Interest,Reconciled Depreciation,Reconciled Cost Of Revenue,...,Policyholder Benefits Gross,Policyholder Benefits Ceded,Occupancy And Equipment,Preferred Stock Dividends,Net Income From Tax Loss Carryforward,Research And Development,Professional Expense And Contract Services Expense,Earnings From Equity Interest Net Of Tax,Net Income Extraordinary,Excise Taxes
0,2023-12-31,24CS,2.478119e+05,0.195597,-44136736.0,1266954.0,1266954.0,-45071044.0,9617938.0,6.714324e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-12-31,24CS,5.479578e+05,0.223307,41876591.0,2453829.0,2453829.0,24494231.0,7726356.0,8.700639e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-12-31,24CS,0.000000e+00,0.241495,35101165.0,0.0,0.0,19455578.0,6959244.0,5.637045e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-12-31,24CS,0.000000e+00,0.352007,18310000.0,NaN,NaN,6940000.0,6310000.0,3.545600e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-12-31,2S,-1.027415e+06,0.037799,248561000.0,-27181000.0,-27181000.0,160083000.0,55027000.0,6.351661e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
incm_stmt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3503 entries, 0 to 3502
Data columns (total 84 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   date                                                        3503 non-null   object 
 1   symbol                                                      3503 non-null   object 
 2   Tax Effect Of Unusual Items                                 3473 non-null   float64
 3   Tax Rate For Calcs                                          3473 non-null   float64
 4   Normalized EBITDA                                           3357 non-null   float64
 5   Total Unusual Items                                         2642 non-null   float64
 6   Total Unusual Items Excluding Goodwill                      2638 non-null   float64
 7   Net Income From Continuing Operation Net Minority Interest  3469 non-null   float64
 8 